In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_claims = pd.read_csv('data/Claims.csv')
df_members = pd.read_csv('data/Members.csv')
df_hospital = pd.read_csv('data/DaysInHospital_Y2.csv')
df_drug = pd.read_csv('data/DrugCount.csv')
df_lab = pd.read_csv('data/LabCount.csv')

In [3]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                  object
LengthOfStay              object
DSFS                      object
PrimaryConditionGroup     object
CharlsonIndex             object
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [4]:
df_claims.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0


In [5]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['count','percent']
    display(null_df)
    
count_nulls(df_claims)

,count,percent
ProviderID,16264,0.609369
Vendor,24856,0.931289
PCP,7492,0.280705
Specialty,8405,0.314913
PlaceSvc,7632,0.285951
LengthOfStay,2597392,97.317412
DSFS,52770,1.977152
PrimaryConditionGroup,11410,0.427503
ProcedureGroup,3675,0.137693


In [6]:
def updatePayDelay():
    df_claims.loc[df_claims['PayDelay']=='162+','PayDelay'] = 162
    df_claims['PayDelay'] = df_claims['PayDelay'].astype(int)
    return

updatePayDelay()

In [7]:
def updateDSFS():
    df_claims.loc[df_claims['DSFS']=='0- 1 month','DSFS'] = 1
    df_claims.loc[df_claims['DSFS']=='1- 2 months','DSFS'] = 2
    df_claims.loc[df_claims['DSFS']=='2- 3 months','DSFS'] = 3
    df_claims.loc[df_claims['DSFS']=='3- 4 months','DSFS'] = 4
    df_claims.loc[df_claims['DSFS']=='4- 5 months','DSFS'] = 5
    df_claims.loc[df_claims['DSFS']=='5- 6 months','DSFS'] = 6
    df_claims.loc[df_claims['DSFS']=='6- 7 months','DSFS'] = 7
    df_claims.loc[df_claims['DSFS']=='7- 8 months','DSFS'] = 8
    df_claims.loc[df_claims['DSFS']=='8- 9 months','DSFS'] = 9
    df_claims.loc[df_claims['DSFS']=='9-10 months','DSFS'] = 10
    df_claims.loc[df_claims['DSFS']=='10-11 months','DSFS'] = 11
    df_claims.loc[df_claims['DSFS']=='11-12 months','DSFS'] = 12
    df_claims['DSFS'].fillna(0, inplace=True)
    df_claims['DSFS'] = df_claims['DSFS'].astype(int)
    return

updateDSFS()

In [8]:
def updateCharlsonIndex():
    df_claims.loc[df_claims['CharlsonIndex']=='0','CharlsonIndex'] = 0
    df_claims.loc[df_claims['CharlsonIndex']=='1-2','CharlsonIndex'] = 2
    df_claims.loc[df_claims['CharlsonIndex']=='3-4','CharlsonIndex'] = 4
    df_claims.loc[df_claims['CharlsonIndex']=='5+','CharlsonIndex'] = 6

    df_claims['CharlsonIndex'] = df_claims['CharlsonIndex'].astype(int)
    return
    
updateCharlsonIndex()

In [9]:
def updateLengthOfStay():
    df_claims.loc[df_claims['LengthOfStay']=='1 day','LengthOfStay'] = 1
    df_claims.loc[df_claims['LengthOfStay']=='2 days','LengthOfStay'] = 2
    df_claims.loc[df_claims['LengthOfStay']=='3 days','LengthOfStay'] = 3
    df_claims.loc[df_claims['LengthOfStay']=='4 days','LengthOfStay'] = 4
    df_claims.loc[df_claims['LengthOfStay']=='5 days','LengthOfStay'] = 5
    df_claims.loc[df_claims['LengthOfStay']=='6 days','LengthOfStay'] = 6
    df_claims.loc[df_claims['LengthOfStay']=='1- 2 weeks','LengthOfStay'] = 11
    df_claims.loc[df_claims['LengthOfStay']=='2- 4 weeks','LengthOfStay'] = 21
    df_claims.loc[df_claims['LengthOfStay']=='4- 8 weeks','LengthOfStay'] = 42
    df_claims.loc[df_claims['LengthOfStay']=='26+ weeks','LengthOfStay'] = 180
    
    df_claims['LengthOfStay'].fillna(0, inplace=True)

    df_claims['LengthOfStay'] = df_claims['LengthOfStay'].astype(int)
    return

updateLengthOfStay()

In [16]:
df_group = df_claims.groupby(['Year','MemberID'])

df_new = pd.DataFrame()
df_new['MemberID'] = df_group['MemberID'].head()
df_new['Year'] = df_group['Year'].head()

df_new['no_Claims'] = df_group['ProviderID'].transform('count')
df_new['no_Providers'] = df_group['ProviderID'].transform('nunique')
df_new['no_Vendors'] = df_group['Vendor'].transform('nunique')
df_new['no_PCPs'] = df_group['PCP'].transform('nunique')
df_new['no_PlaceSvcs'] = df_group['PlaceSvc'].transform('nunique')
df_new['no_Specialities'] = df_group['Specialty'].transform('nunique')
df_new['no_PrimaryConditionGroups'] = df_group['PrimaryConditionGroup'].transform('nunique')
df_new['no_ProcedureGroups'] = df_group['ProcedureGroup'].transform('nunique')

df_new['PayDelay_max'] = df_group['PayDelay'].transform('max')
df_new['PayDelay_min'] = df_group['PayDelay'].transform('min')
df_new['PayDelay_ave'] = df_group['PayDelay'].transform(lambda x: np.average(x))
# df_new['PayDelay_stdev'] = df_group['PayDelay'].transform(lambda x: 1 if x>25 else 0)

df_new.head(10)

,MemberID,Year,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,no_PrimaryConditionGroups,no_ProcedureGroups,PayDelay_max,PayDelay_min,PayDelay_ave
0,42286978,Y1,12,8.0,8.0,1.0,3,5,8,5,114,27,49.416667
1,97903248,Y3,44,11.0,11.0,1.0,3,7,8,7,135,0,27.750000
2,2759427,Y3,3,2.0,2.0,1.0,2,2,1,3,31,14,25.333333
3,73570559,Y3,30,8.0,5.0,1.0,2,5,7,6,86,15,34.733333
4,11837054,Y2,43,7.0,6.0,1.0,3,6,7,5,42,15,29.116279
5,45844561,Y3,6,1.0,1.0,1.0,1,1,2,1,25,17,20.166667
6,99829076,Y1,12,5.0,4.0,1.0,2,5,4,5,162,23,106.000000
7,54666321,Y1,36,8.0,6.0,1.0,2,6,5,6,162,20,57.750000
8,60497718,Y2,15,6.0,6.0,1.0,3,4,8,5,162,21,45.066667
9,72200595,Y3,44,16.0,12.0,1.0,6,8,14,7,162,0,51.954545


In [ ]:
df_new['pgc1'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MSC2a3'))
df_new['pgc2'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='METAB3'))
df_new['pgc3'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ARTHSPIN'))
df_new['pgc4'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='NEUMENT'))
df_new['pgc5'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RESPR4'))
df_new['pgc6'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCHRT'))
df_new['pgc7'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SKNAUT'))
df_new['pgc8'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GIBLEED'))
df_new['pgc9'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='INFEC4'))
df_new['pgc10'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='TRAUMA'))
df_new['pgc11'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEART2'))
df_new['pgc12'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL3'))
df_new['pgc13'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ROAMI'))
df_new['pgc14'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCL5'))
df_new['pgc15'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='ODaBNCA'))
df_new['pgc16'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='UTI'))
df_new['pgc17'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='COPD'))
df_new['pgc18'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GYNEC1'))
df_new['pgc19'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRB'))
df_new['pgc20'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='FXDISLC'))
df_new['pgc21'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='AMI'))
df_new['pgc22'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PRGNCY'))
df_new['pgc23'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEMTOL'))
df_new['pgc24'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HEART4'))
df_new['pgc25'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SEIZURE'))
df_new['pgc26'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='APPCHOL'))
df_new['pgc27'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CHF'))
df_new['pgc28'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GYNECA'))
df_new['pgc29'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x.isnull()))
df_new['pgc30'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PNEUM'))
df_new['pgc31'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL2'))
df_new['pgc32'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='GIOBSENT'))
df_new['pgc33'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='STROKE'))
df_new['pgc34'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRA'))
df_new['pgc35'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='FLaELEC'))
df_new['pgc36'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='MISCL1'))
df_new['pgc37'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='HIPFX'))
df_new['pgc38'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='METAB1'))
df_new['pgc39'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PERVALV'))
df_new['pgc40'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='LIVERDZ'))
df_new['pgc41'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CATAST'))
df_new['pgc42'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='CANCRM'))
df_new['pgc43'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PERINTL'))
df_new['pgc44'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='PNCRDZ'))
df_new['pgc45'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='RENAL1'))
df_new['pgc46'] = df_group['PrimaryConditionGroup'].transform(lambda x: sum(x=='SEPSIS'))

In [19]:
df_new.head(10)

,MemberID,Year,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,no_PrimaryConditionGroups,no_ProcedureGroups,...,pgc13,pgc14,pgc15,pgc16,pgc17,pgc18,pgc19,pgc20,pgc21,pgc22
0,42286978,Y1,12,8.0,8.0,1.0,3,5,8,5,...,0,0,1,0,0,0,0,0,0,0
1,97903248,Y3,44,11.0,11.0,1.0,3,7,8,7,...,0,0,3,0,0,0,0,0,0,0
2,2759427,Y3,3,2.0,2.0,1.0,2,2,1,3,...,0,0,0,0,0,0,0,0,0,0
3,73570559,Y3,30,8.0,5.0,1.0,2,5,7,6,...,0,0,0,0,0,2,0,0,0,0
4,11837054,Y2,43,7.0,6.0,1.0,3,6,7,5,...,0,0,0,0,4,0,0,15,0,0
5,45844561,Y3,6,1.0,1.0,1.0,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
6,99829076,Y1,12,5.0,4.0,1.0,2,5,4,5,...,0,0,0,0,0,0,0,0,0,0
7,54666321,Y1,36,8.0,6.0,1.0,2,6,5,6,...,0,0,0,0,1,0,0,0,0,0
8,60497718,Y2,15,6.0,6.0,1.0,3,4,8,5,...,0,1,1,0,1,0,0,0,0,0
9,72200595,Y3,44,16.0,12.0,1.0,6,8,14,7,...,0,1,4,2,0,0,0,0,0,0


In [ ]:
df_new['sp1'] = df_group['Specialty'].transform(lambda x: sum(x=='Internal'))
df_new['sp2'] = df_group['Specialty'].transform(lambda x: sum(x=='Laboratory'))
df_new['sp3'] = df_group['Specialty'].transform(lambda x: sum(x=='General Practice'))
df_new['sp4'] = df_group['Specialty'].transform(lambda x: sum(x=='Surgery'))
df_new['sp5'] = df_group['Specialty'].transform(lambda x: sum(x=='Diagnostic Imaging'))
df_new['sp6'] = df_group['Specialty'].transform(lambda x: sum(x=='Emergency'))
df_new['sp7'] = df_group['Specialty'].transform(lambda x: sum(x=='Other'))
df_new['sp8'] = df_group['Specialty'].transform(lambda x: sum(x=='Pediatrics'))
df_new['sp9'] = df_group['Specialty'].transform(lambda x: sum(x=='Rehabilitation'))
df_new['sp10'] = df_group['Specialty'].transform(lambda x: sum(x=='Obstetrics and Gynecology'))
df_new['sp11'] = df_group['Specialty'].transform(lambda x: sum(x=='Anesthesiology'))
df_new['sp12'] = df_group['Specialty'].transform(lambda x: sum(x=='Pathology'))
df_new['sp13'] = df_group['Specialty'].transform(lambda x: sum(x.isnull()))

In [ ]:
df_new['pg1'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='EM'))
df_new['pg2'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='PL'))
df_new['pg3'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='MED'))                                                                 
df_new['pg4'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SCS'))
df_new['pg5'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='RAD'))
df_new['pg6'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SDS'))
df_new['pg7'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SIS'))
df_new['pg8'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SMS'))                                                                 
df_new['pg9'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='ANES'))
df_new['pg10'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SGS'))
df_new['pg11'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SEOA'))
df_new['pg12'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SRS'))
df_new['pg13'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SNS'))                                                                 
df_new['pg14'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SAS'))
df_new['pg15'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SUS'))
df_new['pg16'] = df_group['ProcedureGroup'].transform(lambda x: sum(x.isnull()))
df_new['pg17'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SMCD'))
df_new['pg18'] = df_group['ProcedureGroup'].transform(lambda x: sum(x=='SO'))

In [ ]:
df_new['ps1'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Office'))                   
df_new['ps2'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Independent Lab'))                                             
df_new['ps3'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Urgent Care'))                                                             
df_new['ps4'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Outpatient Hospital'))                                                                
df_new['ps5'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Inpatient Hospital'))                                                                 
df_new['ps6'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Ambulance'))                                                                 
df_new['ps7'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Other'))                                                                 
df_new['ps8'] = df_group['PlaceSvc'].transform(lambda x: sum(x=='Home'))                                                                 
df_new['ps9'] = df_group['PlaceSvc'].transform(lambda x: sum(x.isnull()))  

In [4]:
df_members.head()

,MemberID,AgeAtFirstClaim,Sex
0,14723353,70-79,M
1,75706636,70-79,M
2,17320609,70-79,M
3,69690888,40-49,M
4,33004608,0-9,M


In [5]:
df_members.dtypes

MemberID            int64
AgeAtFirstClaim    object
Sex                object
dtype: object

In [7]:
df_new_members = pd.DataFrame()
df_new_members['MemberID']=df_members['MemberID']

In [9]:
def updateAge():
    df_new_members['age_05'] = np.where(df_members['AgeAtFirstClaim']=='0-9', 1, 0)
    df_new_members['age_15'] = np.where(df_members['AgeAtFirstClaim']=='10-19', 1, 0)
    df_new_members['age_25'] = np.where(df_members['AgeAtFirstClaim']=='20-29', 1, 0)
    df_new_members['age_35'] = np.where(df_members['AgeAtFirstClaim']=='30-39', 1, 0)
    df_new_members['age_45'] = np.where(df_members['AgeAtFirstClaim']=='40-49', 1, 0)
    df_new_members['age_55'] = np.where(df_members['AgeAtFirstClaim']=='50-59', 1, 0)
    df_new_members['age_65'] = np.where(df_members['AgeAtFirstClaim']=='60-69', 1, 0)
    df_new_members['age_75'] = np.where(df_members['AgeAtFirstClaim']=='70-79', 1, 0)
    df_new_members['age_85'] = np.where(df_members['AgeAtFirstClaim']=='80+', 1, 0)
    df_new_members['age_MISS'] = np.where(df_members['AgeAtFirstClaim'].isnull(), 1, 0)
    return

updateAge()
df_new_members.head()

,MemberID,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,age_85,age_MISS
0,14723353,0,0,0,0,0,0,0,1,0,0
1,75706636,0,0,0,0,0,0,0,1,0,0
2,17320609,0,0,0,0,0,0,0,1,0,0
3,69690888,0,0,0,0,1,0,0,0,0,0
4,33004608,1,0,0,0,0,0,0,0,0,0


In [12]:
def updateSex():
    df_new_members['SexMALE'] = np.where(df_members['Sex']=='M', 1, 0)
    df_new_members['sexFEMALE'] = np.where(df_members['Sex']=='F', 1, 0)
    df_new_members['sexMISS'] = np.where(df_members['Sex'].isnull(), 1, 0)
    return

updateSex()
df_new_members.dtypes
df_new_members.head()

,MemberID,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS
0,14723353,0,0,0,0,0,0,0,1,0,0,1,0,0
1,75706636,0,0,0,0,0,0,0,1,0,0,1,0,0
2,17320609,0,0,0,0,0,0,0,1,0,0,1,0,0
3,69690888,0,0,0,0,1,0,0,0,0,0,1,0,0
4,33004608,1,0,0,0,0,0,0,0,0,0,1,0,0


In [13]:
df_new_members.dtypes

MemberID     int64
age_05       int32
age_15       int32
age_25       int32
age_35       int32
age_45       int32
age_55       int32
age_65       int32
age_75       int32
age_85       int32
age_MISS     int32
SexMALE      int32
sexFEMALE    int32
sexMISS      int32
dtype: object

In [14]:
df_drug.head()

,MemberID,Year,DSFS,DrugCount
0,48925661,Y2,9-10 months,7+
1,90764620,Y3,8- 9 months,3
2,61221204,Y1,2- 3 months,1
3,63628544,Y3,1- 2 months,1
4,46949606,Y2,10-11 months,3


In [18]:
count_nulls(df_drug)

,count,percent


In [20]:
def updateDrugCount():
    df_drug.loc[df_drug['DrugCount']=='7+','DrugCount'] = 7
    
    df_drug['DrugCount'] = df_drug['DrugCount'].astype(int)
    return

updateDrugCount()
df_drug.dtypes

MemberID      int64
Year         object
DSFS         object
DrugCount     int32
dtype: object

In [23]:
df_group_drug = df_drug.groupby(['Year','MemberID'])

df_new_drug = pd.DataFrame()
df_new_drug['MemberID'] = df_group_drug['MemberID'].head()
df_new_drug['Year'] = df_group_drug['Year'].head()

df_new_drug['DrugCount_max'] = df_group_drug['DrugCount'].transform('max')
df_new_drug['DrugCount_min'] = df_group_drug['DrugCount'].transform('min')
df_new_drug['DrugCount_ave'] = df_group_drug['DrugCount'].transform(lambda x: np.average(x))
df_new_drug['DrugCount_months'] = df_group_drug['MemberID'].transform('count')

In [24]:
df_new_drug.head()

,MemberID,Year,DrugCount_max,DrugCount_min,DrugCount_ave,Drugcount_months
0,48925661,Y2,7,2,5.916667,12
1,90764620,Y3,5,1,3.000000,4
2,61221204,Y1,3,1,1.833333,6
3,63628544,Y3,2,1,1.500000,2
4,46949606,Y2,5,1,2.818182,11


In [19]:
df_lab.head()

,MemberID,Year,DSFS,LabCount
0,69258001,Y3,2- 3 months,1
1,10143167,Y1,0- 1 month,2
2,1054357,Y1,0- 1 month,6
3,56583841,Y3,6- 7 months,4
4,70967047,Y2,0- 1 month,2


In [21]:
df_lab.dtypes

MemberID     int64
Year        object
DSFS        object
LabCount     int32
dtype: object

In [18]:
def updateLabCount():
    df_lab.loc[df_lab['LabCount']=='10+','LabCount'] = 10
    
    df_lab['LabCount'] = df_lab['LabCount'].astype(int)
    return
updateLabCount()
df_lab.dtypes

MemberID     int64
Year        object
DSFS        object
LabCount     int32
dtype: object

In [22]:
df_group_lab = df_lab.groupby(['Year','MemberID'])

df_new_lab = pd.DataFrame()
df_new_lab['MemberID'] = df_group_lab['MemberID'].head()
df_new_lab['Year'] = df_group_lab['Year'].head()

df_new_lab['LabCount_max'] = df_group_lab['LabCount'].transform('max')
df_new_lab['LabCount_min'] = df_group_lab['LabCount'].transform('min')
df_new_lab['LabCount_ave'] = df_group_lab['LabCount'].transform(lambda x: np.average(x))
df_new_lab['LabCount_months'] = df_group_lab['MemberID'].transform('count')

In [23]:
df_new_lab.head()

,MemberID,Year,LabCount_max,LabCount_min,LabCount_ave,LabCount_months
0,69258001,Y3,5,1,2.5,4
1,10143167,Y1,6,2,4.0,2
2,1054357,Y1,6,1,4.5,4
3,56583841,Y3,7,4,5.5,2
4,70967047,Y2,10,1,2.9,10


In [24]:
df_new_lab.to_csv('data/new_lab.csv', sep='\t')